# 🏎️ DAC-004 Lesson 2 — Intermediate SQL (F1 Edition)
**Instructor Version** with solutions.

In [ ]:
# Setup: install and load SQL magic, create in-memory DB
!pip install ipython-sql sqlalchemy prettytable -q
%load_ext sql
%sql sqlite://
%config SqlMagic.style = 'PLAIN_COLUMNS'
%config SqlMagic.autopandas = True

%%sql
DROP TABLE IF EXISTS Team;
DROP TABLE IF EXISTS Driver;
DROP TABLE IF EXISTS Race;
DROP TABLE IF EXISTS Result;

CREATE TABLE Team (
  TeamID INTEGER PRIMARY KEY,
  Name TEXT,
  BaseCountry TEXT
);

CREATE TABLE Driver (
  DriverID INTEGER PRIMARY KEY,
  Name TEXT,
  Age INTEGER,
  Nationality TEXT,
  TeamID INTEGER,
  FOREIGN KEY(TeamID) REFERENCES Team(TeamID)
);

CREATE TABLE Race (
  RaceID INTEGER PRIMARY KEY,
  Name TEXT,
  Location TEXT,
  Date TEXT
);

CREATE TABLE Result (
  DriverID INTEGER,
  RaceID INTEGER,
  Position INTEGER,
  Points INTEGER,
  PRIMARY KEY (DriverID, RaceID),
  FOREIGN KEY(DriverID) REFERENCES Driver(DriverID),
  FOREIGN KEY(RaceID) REFERENCES Race(RaceID)
);

INSERT INTO Team VALUES
(1, 'Red Bull Racing', 'Austria'),
(2, 'Ferrari', 'Italy'),
(3, 'Mercedes', 'UK'),
(4, 'McLaren', 'UK'),
(5, 'Aston Martin', 'UK'),
(6, 'Alpine', 'France');

INSERT INTO Driver VALUES
(1, 'Max Verstappen', 28, 'Netherlands', 1),
(2, 'Daniel Ricciardo', 36, 'Australia', 1),
(3, 'Charles Leclerc', 27, 'Monaco', 2),
(4, 'Carlos Sainz', 30, 'Spain', 2),
(5, 'Lewis Hamilton', 40, 'UK', 3),
(6, 'George Russell', 27, 'UK', 3),
(7, 'Lando Norris', 26, 'UK', 4),
(8, 'Oscar Piastri', 24, 'Australia', 4),
(9, 'Fernando Alonso', 43, 'Spain', 5),
(10, 'Pierre Gasly', 29, 'France', 6);

INSERT INTO Race VALUES
(1, 'Bahrain Grand Prix', 'Sakhir', '2024-03-02'),
(2, 'Monaco Grand Prix', 'Monte Carlo', '2024-05-26'),
(3, 'British Grand Prix', 'Silverstone', '2024-07-07'),
(4, 'Austrian Grand Prix', 'Spielberg', '2024-06-30'),
(5, 'Singapore Grand Prix', 'Marina Bay', '2024-09-22');

INSERT INTO Result VALUES
(1,1,1,25),(2,1,4,12),(3,1,3,15),(5,1,2,18),(7,1,5,10),
(1,2,1,25),(8,2,2,18),(3,2,3,15),(5,2,4,12),(9,2,5,10),
(5,3,1,25),(1,3,2,18),(7,3,3,15),(6,3,4,12),(3,3,5,10),
(1,4,1,25),(2,4,3,15),(7,4,2,18),(10,4,5,10),(9,4,4,12),
(7,5,1,25),(1,5,2,18),(8,5,3,15),(3,5,4,12),(4,5,5,10);


## 0) Recap

In [ ]:
%%sql
SELECT Name FROM Driver WHERE TeamID = 1;

In [ ]:
%%sql
SELECT d.Name AS Driver, t.Name AS Team
FROM Driver d JOIN Team t ON d.TeamID = t.TeamID
ORDER BY Team, Driver;

## 1) INSERT

In [ ]:
%%sql
INSERT INTO Driver (DriverID, Name, Age, Nationality, TeamID)
VALUES (11, 'Liam Lawson', 23, 'New Zealand', 6);

SELECT * FROM Driver ORDER BY DriverID;

## 2) UPDATE

In [ ]:
%%sql
UPDATE Driver SET TeamID = 4 WHERE Name = 'Daniel Ricciardo';
SELECT Name, TeamID FROM Driver WHERE Name = 'Daniel Ricciardo';

## 3) DELETE

In [ ]:
%%sql
DELETE FROM Race WHERE Name = 'Monaco Grand Prix';
SELECT * FROM Race ORDER BY Date;

## 4) DISTINCT

In [ ]:
%%sql
SELECT DISTINCT BaseCountry FROM Team ORDER BY BaseCountry;

## 5) LIMIT & OFFSET

In [ ]:
%%sql
SELECT d.Name, SUM(r.Points) AS TotalPoints
FROM Result r JOIN Driver d ON r.DriverID = d.DriverID
GROUP BY d.Name
ORDER BY TotalPoints DESC
LIMIT 3;

In [ ]:
%%sql
SELECT d.Name, SUM(r.Points) AS TotalPoints
FROM Result r JOIN Driver d ON r.DriverID = d.DriverID
GROUP BY d.Name
ORDER BY TotalPoints DESC
LIMIT 3 OFFSET 3;

## 6) BETWEEN

In [ ]:
%%sql
SELECT Name, Date FROM Race
WHERE Date BETWEEN '2024-05-01' AND '2024-08-01'
ORDER BY Date;

## 7) CASE WHEN

In [ ]:
%%sql
SELECT d.Name,
       SUM(r.Points) AS TotalPoints,
       CASE 
         WHEN SUM(r.Points) >= 70 THEN 'Elite'
         WHEN SUM(r.Points) >= 45 THEN 'Strong'
         ELSE 'Rookie'
       END AS PerformanceLevel
FROM Result r JOIN Driver d ON r.DriverID = d.DriverID
GROUP BY d.Name
ORDER BY TotalPoints DESC;

## 8) JOINS Deep Dive

In [ ]:
%%sql
SELECT dr.Name AS Driver, te.Name AS Team, ra.Name AS Race, r.Position, r.Points
FROM Result r
JOIN Driver dr ON r.DriverID = dr.DriverID
JOIN Team te ON dr.TeamID = te.TeamID
JOIN Race ra ON r.RaceID = ra.RaceID
ORDER BY ra.Date, r.Position;

In [ ]:
%%sql
SELECT te.Name AS Team, dr.Name AS Driver
FROM Team te
LEFT JOIN Driver dr ON dr.TeamID = te.TeamID
ORDER BY Team, Driver;

In [ ]:
%%sql
SELECT te.Name AS Team, dr.Name AS Driver
FROM Team te LEFT JOIN Driver dr ON dr.TeamID = te.TeamID
UNION
SELECT te.Name AS Team, dr.Name AS Driver
FROM Driver dr LEFT JOIN Team te ON dr.TeamID = te.TeamID
ORDER BY Team, Driver;

## 9) Subqueries & Derived Tables

In [ ]:
%%sql
WITH driver_totals AS (
  SELECT DriverID, SUM(Points) AS Total FROM Result GROUP BY DriverID
)
SELECT d.Name, dt.Total
FROM driver_totals dt JOIN Driver d ON dt.DriverID = d.DriverID
WHERE dt.Total > (SELECT AVG(Total) FROM driver_totals)
ORDER BY dt.Total DESC;

## 10) Transactions

In [ ]:
%%sql
BEGIN;
UPDATE Result SET Points = Points + 5
WHERE DriverID IN (SELECT DriverID FROM Driver WHERE TeamID = 4);
SELECT d.Name, SUM(r.Points) AS TotalPoints
FROM Result r JOIN Driver d ON r.DriverID = d.DriverID
GROUP BY d.Name ORDER BY TotalPoints DESC;
ROLLBACK;

## 11) Leaderboards

In [ ]:
%%sql
SELECT d.Name, SUM(r.Points) AS TotalPoints
FROM Result r JOIN Driver d ON r.DriverID = d.DriverID
GROUP BY d.Name
ORDER BY TotalPoints DESC;

In [ ]:
%%sql
SELECT t.Name AS Team, SUM(r.Points) AS TeamPoints
FROM Result r
JOIN Driver d ON r.DriverID = d.DriverID
JOIN Team t ON d.TeamID = t.TeamID
GROUP BY t.TeamID
ORDER BY TeamPoints DESC;

## 12) Challenges
1) Insert 'Japanese Grand Prix' on '2024-04-07' at 'Suzuka'.
2) Increase Ferrari drivers' points by 10% (round down).
3) Delete results with points < 12.
4) Show average points per race per team.
5) Top 3 drivers by points after deletions.